In [1]:
from nocturne.envs.nocturne_gymnasium import NocturneGymnasium, CustomPostprocessor
import yaml
from nocturne.envs.base_env import BaseEnv
from nocturne.envs.vec_env_ma import MultiAgentAsVecEnv

import pufferlib.vectorization
vec = pufferlib.vectorization.Multiprocessing
# vec = pufferlib.vectorization.Serial

import pufferlib.emulation
import pufferlib.wrappers

from time import perf_counter

# Load environment settings
with open(f"../configs/env_config.yaml", "r") as stream:
    env_config = yaml.safe_load(stream)

In [2]:
def make_env(env_config):
    return NocturneGymnasium(config=env_config, num_agents=env_config["max_num_vehicles"]) 

def nocturne_creator(env_config):
    return pufferlib.emulation.GymnasiumPufferEnv(env_creator=make_env, env_args=(env_config,), postprocessor_cls=CustomPostprocessor)

In [3]:
# envs = vec(nocturne_creator,env_args=[env_config,], num_envs=4, envs_per_worker=2, env_pool=True)

In [4]:
# envs.async_reset()
# obs = envs.recv()[0]

In [5]:
# actions = [envs.single_action_space.sample() for _ in range(4)]
# envs.step(actions)

In [6]:
NUM_STEPS = 1000

env = make_env(env_config)
env.reset()

total_time = 0

for i in range(NUM_STEPS):
    actions = env.action_space.sample()
    start = perf_counter()
    env.step(actions)
    end = perf_counter()
    total_time += end - start

print(f"Average time for single env step: {total_time/NUM_STEPS}")
print(f"Average FPS for single env step: {NUM_STEPS/(total_time)}")

Average time for single env step: 0.002120851782616228
Average FPS for single env step: 471.50866844944056


In [7]:
NUM_STEPS = 1000
NUM_PROCESSES = 32
NUM_ENVS = 128
assert(NUM_ENVS % NUM_PROCESSES == 0)
NUM_ENVS_PER_WORKER = NUM_ENVS // NUM_PROCESSES

envs = vec(nocturne_creator,env_args=[env_config,], num_envs=NUM_ENVS, envs_per_worker=1, env_pool=False)
envs.async_reset()
obs = envs.recv()[0]

total_time = 0
for i in range(NUM_STEPS):
    actions = [envs.single_action_space.sample() for _ in range(NUM_ENVS)]
    start = perf_counter()
    envs.step(actions)
    end = perf_counter()
    total_time += end - start
envs.close()
print(f"Average time for {NUM_STEPS} env step: {total_time/(NUM_STEPS*NUM_ENVS)}")
print(f"Average FPS for {NUM_STEPS} env step: {(NUM_STEPS/(total_time))*NUM_ENVS}")

Average time for 1000 env step: 0.0010244677745140506
Average FPS for 1000 env step: 976.1165991525143
